In [1]:
# Imports
from subsbml import System, createSubsystem, combineSystems, createNewSubsystem
import numpy as np
import pylab as plt

from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import bokeh.io
import bokeh.plotting

## Cell 1 simple rR1 transport w/ rR1 external reservoir

In [2]:
ss1 = createSubsystem ("BiSwitch_CRN.xml")
ss2 = createSubsystem ("rR1_external_reservoir.xml")

# Create a simple rR1 membrane 
mb1 = createSubsystem("membrane_rR1.xml", membrane = True)

cell_1 = System("cell_1")

cell_1.setInternal([ss1])
cell_1.setExternal([ss2])
cell_1.setMembrane(mb1)

cell_1_model = cell_1.getModel()


#cell_1 = System ("cell_1", ListOfInternalSubsystems = [ss1], 
#                 ListOfExternalSubsystems = [ss2], 
#                 ListOfMembraneSubsystems = [mb1])

##### Set Species Amount #####

cell_1_model.setSpeciesAmount('rR1_e', 1e4, compartment = 'cell_1_external')
cell_1_model.setSpeciesAmount('Core1_OFF', 6e3, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('Core2_OFF', 5e3, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('dna_dA2', 6e3, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('dna_dA1', 6e3, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('protein_RNAseH', 25, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('protein_RNAP', 150, compartment = 'cell_1_internal')
cell_1_model.getSBMLDocument().getModel().getCompartment(1).setSize(1e-4) 

cell_1_model.writeSBML('cell_1_model.xml')


The subsystem from membrane_rR1.xml has multiple compartments


1

In [3]:
# Calling Names

X_id1 = cell_1_model.getSpeciesByName('complex_Core1_ON').getId()
X_id2 = cell_1_model.getSpeciesByName('Core1_OFF').getId()
X_id3 = cell_1_model.getSpeciesByName('complex_Core2_ON').getId()
X_id4 = cell_1_model.getSpeciesByName('Core2_OFF').getId()
X_id5 = cell_1_model.getSpeciesByName("rna_rR2").getId()
X_id6 = cell_1_model.getSpeciesByName("rna_rR1_i").getId()
X_id7 = cell_1_model.getSpeciesByName("rR1_e").getId()

# Simulate with BioScrape
timepoints = np.linspace(0, 86400, 1000)
timepoints = timepoints/3600
results_1,_ = cell_1_model.simulateWithBioscrape(timepoints)

# For label convenience
x = 'Time (hours)'
y = 'Concentration (uM)'

bokeh.io.output_notebook()
a = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
a.circle(timepoints, results_1[X_id1], legend_label = "Core1 ON" , color = "red")
a.circle(timepoints, results_1[X_id3], legend_label = "Core2 ON", color = "blue")
a.legend.click_policy="hide"
a.legend.location="bottom_right"

b = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
b.circle(timepoints, results_1[X_id5], legend_label = "rR2" , color = "red")
b.circle(timepoints, results_1[X_id6], legend_label = "rR1", color = "blue")
#b.circle(timepoints, results_1[X_id7], legend_label = "rR1_e", color = "orange")
b.legend.click_policy="hide"
b.legend.location="bottom_right"

bokeh.io.show(row(a, b))
warnings.filterwarnings("ignore")


C:\Users\Jeremiah\anaconda3\lib\site-packages\bioscrape\sbmlutil.py:93: UserWarning: SBML model contains reversible reaction!
Please check rate expressions and ensure they are non-negative before doing stochastic simulations.
  'Please check rate expressions and ensure they are non-negative before doing '+
C:\Users\Jeremiah\anaconda3\lib\site-packages\bioscrape\sbmlutil.py:208: UserWarning: Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. Refer to the bioscrape wiki for more information.
  warnings.warn('Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. ' +


Loading BokehJS ...

**Notes**

Simple membrane implemented with diffusion of rR1 species.

Bistability is not being shown by either of the cores. RNA production very low. Possible bug in Sub-sbml(?)